# Processing IV/dIdV sweeps with `pytesdaq`
-----
The processing will extract relevant features and place them in a pandas DataFrame for further post-processing by the IVanalysis class.

__Authors__: Caleb Fink, Samuel Watkins

In [1]:
import pandas as pd
import time as tm
from glob import glob

from pytesdaq.processing import process_ivsweep

In [2]:
savepath = '/where/the/processed/data/should/be/saved/'
sweeppath = '/Folder/where/all/there/series/for/IV/sweep/are/saved/'
savename = 'name_to_save_DF_as'
chans = ['PES2']

In [3]:
nfiles = len(glob(f'{sweeppath}*'))
print(f"There are {nfiles} files.")

There are 66 files.


In [4]:
start_time = tm.time()
df = process_ivsweep(
    ivfilepath=sweeppath,
    chans=chans,
    lgcverbose=False, # Switch to True to know which file its on
    lgcsave=True,
    savepath=savepath,
    savename=savename,
)
end_time = tm.time()

print(f'IV sweep took {(end_time - start_time)/60: .3f} min to process')

IV sweep took  23.042 min to process


## Post-process the DataFrame with `IVanalysis`

In [6]:
from pytesdaq.processing import IVanalysis

To post-process the saved dataframe one must first load the saved `pickle` file. It is not recommended to do this in the same notebook as the processing step above, as the above, in principle, should only be run once.

To load via `pickle` and start the post-processing, the user may take advantage of Pandas:

In [7]:
df = pd.read_pickle('/path/to/where/the/processed/data/was/stored/file.pkl')
    
IVobj = IVanalysis(
    df, 
    nnorm=5, # number of points in normal region
    nsc=5, # number of points in superconducting region
)

In [8]:
IVanalysis.__init__?

Signature:
IVanalysis.__init__(
    self,
    df,
    nnorm,
    nsc,
    ntran=None,
    channels=None,
    channelname='',
    rshunt=0.005,
    rshunt_err=0.00025,
    tbath=0,
    tbath_err=0,
    tc=0,
    tc_err=0,
    Gta=0,
    Gta_err=0,
    ib_err=None,
    lgcremove_badseries=True,
    figsavepath='',
)
Docstring:
Initialization of IVanalysis object. Note, currently only
single channel analysis is supported.

Parameters
----------
df : Pandas.core.DataFrame
    DataFrame of a processed IV/dIdV sweep returned from 
    rqpy._process_iv_didv.process_ivsweep().
nnorm : int
    Number bias values where the TES was normal,
    Note: count only one per noise and didv point (don't double
    count!)
nsc : int
    Number of bias values where the TES was Super Conducting,
    Note: count only one per noise and didv point (don't double
    count!)
ntran : range object, NoneType, optional
    The range of the transition data points.
    If ntran is None, then it is left as the total-(n

The general order of each of the class's methods usually follows as below. When doing this analysis, the user should read the docstrings and understand all of the optional arguments. Otherwise the outputs may be nonsensical, as each analysis is unique.

In [9]:
# IV curves
IVobj.analyze_sweep()
# dIdV fitting
IVobj.fit_rload_rn()
IVobj.fit_tran_didv()
# noise modeling
IVobj.fit_normal_noise()
IVobj.fit_sc_noise()
IVobj.model_noise_simple()
# estimate expected baseline energy resolution
IVobj.estimate_noise_errors()
IVobj.find_optimum_bias()